In [4]:
import re
import time
import requests
from bs4 import BeautifulSoup

In [3]:
# open web page

# while link anchor does not have 1994:
### scrape the link
### if not every link has 1994 then increment and go to next page. 
### if every link has 1994 end scraping. 


# https://www.congress.gov/search?q=



# https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22war%22%7D&pageSize=100&page=1
# https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22change%22%7D&pageSize=100&page=1

#https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22climate%20change%22%7D
#https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22natural%20gas%22%7D


#if year is not 1994, then increment (go to page 2)




In [46]:
def scrape_links(url):
    daily_editions = re.compile(r'20.*/|199[5-9]/')
    bound_editions = re.compile(r'/1990/|/1991/|/1992/|/1993/|/1994/')

    bound_edition_hits = 0
    page = 14

    urls = {}
    
    while bound_edition_hits < 100:
        page = page + 1

        #print('page' + str(page))

        request = requests.get(url + str(page))
        current_page = BeautifulSoup(request.text, 'html.parser')

        all_links_current_page = current_page.find_all('a')

        for link in all_links_current_page:
            if 'congressional-record' in link.get('href'):
                
                if re.search(bound_editions, link.get('href')):
                    bound_edition_hits = bound_edition_hits + 1

                    #print(link.get('href'))

                elif re.search(daily_editions, link.get('href')):
                    result = re.search(r'(?<=").*(?=")', str(link))
                    urls['https://www.congress.gov' + str(result.group(0))] = 'url'


        #print('beh' + str(bound_edition_hits))

        time.sleep(1)

    return(urls)

In [53]:
def scrape_record(all_links):
    match_target_text = re.compile(r'(?<=From the Congressional Record Online through the Government Publishing Office \[www.gpo.gov\] ).*(?=____________________)', re.MULTILINE)

    target_text = {}

    for link in all_links.keys():
        request = requests.get(link)
        current_page = BeautifulSoup(request.text, 'html.parser')

        untagged_text = current_page.body.find_all(text=True)

        for text in untagged_text:
            text = text.replace('\n',' ')
            text = " ".join(text.split())

            matched = re.search(match_target_text, text).group(0)
            
            if matched is not None:
                target_text[matched] = 'target_text'
            else:
                pass
        
        time.sleep(1)
    
    return(target_text)
    


In [54]:
def format_record(record):
    for full_record in record.keys():
        title = re.match('([A-Z]+)', full_record)
        print(title)

In [48]:
test = scrape_links('https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22croatian%22%7D&pageSize=100&page=')

In [50]:
test_2 = scrape_record(test)

In [52]:
test_2

{"SENDING AMERICAN TROOPS TO BOSNIA Mr. INHOFE. Mr. President, I feel compelled today to make a couple of statements about the President's message last night. I am very disturbed at what is happening, and I think all of America needs to know what is going on. I commend the President on giving a beautiful, persuasive speech, as he is very good at doing. However, I suggest, Mr. President, that as we are speaking now and as time is creeping by, our troops are on their way to Bosnia. It is my understanding that the distinguished Senator from Colorado, who will be here in just a moment, made a trip over Thanksgiving, which is essentially the same trip I made the week before, into the northeast sector of Bosnia, which is the area where our troops are going to be. A number of people have gone over to Bosnia but have not gone beyond Sarajevo and do not really have a feel for the environment in which our President has this obsession of sending our American troops. Mr. President, last night he t